In [1]:
import pandas as pd

In [2]:
file_path = "../data/raw/energy_prices_non_household.csv"
energy_df = pd.read_csv(file_path)

In [3]:
energy_df.head()

,DATAFLOW,LAST UPDATE,freq,product,nrg_cons,unit,tax,currency,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:NRG_PC_205$DEFAULTVIEW(1.0),22/10/25 23:00:00,"Half-yearly, semesterly",Electrical energy,Consumption from 500 MWh to 1 999 MWh - band IC,Kilowatt-hour,All taxes and levies included,Euro,Albania,2020-S2,0.1243,e,NaN
1,ESTAT:NRG_PC_205$DEFAULTVIEW(1.0),22/10/25 23:00:00,"Half-yearly, semesterly",Electrical energy,Consumption from 500 MWh to 1 999 MWh - band IC,Kilowatt-hour,All taxes and levies included,Euro,Albania,2021-S1,0.1247,e,NaN
2,ESTAT:NRG_PC_205$DEFAULTVIEW(1.0),22/10/25 23:00:00,"Half-yearly, semesterly",Electrical energy,Consumption from 500 MWh to 1 999 MWh - band IC,Kilowatt-hour,All taxes and levies included,Euro,Albania,2021-S2,0.1263,e,NaN
3,ESTAT:NRG_PC_205$DEFAULTVIEW(1.0),22/10/25 23:00:00,"Half-yearly, semesterly",Electrical energy,Consumption from 500 MWh to 1 999 MWh - band IC,Kilowatt-hour,All taxes and levies included,Euro,Albania,2022-S1,0.1268,NaN,NaN
4,ESTAT:NRG_PC_205$DEFAULTVIEW(1.0),22/10/25 23:00:00,"Half-yearly, semesterly",Electrical energy,Consumption from 500 MWh to 1 999 MWh - band IC,Kilowatt-hour,All taxes and levies included,Euro,Albania,2022-S2,0.1316,NaN,NaN


In [4]:
energy_df.columns

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'product', 'nrg_cons', 'unit', 'tax',
       'currency', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG',
       'CONF_STATUS'],
      dtype='object')

In [5]:
energy_df.shape

(3501, 13)

In [6]:
energy_df["geo"].value_counts().head(10)

geo
Albania                   90
Austria                   90
Bosnia and Herzegovina    90
Belgium                   90
Bulgaria                  90
Cyprus                    90
Czechia                   90
Germany                   90
Denmark                   90
Estonia                   90
Name: count, dtype: int64

In [7]:
energy_df["nrg_cons"].value_counts().head(10)

nrg_cons
Consumption from 500 MWh to 1 999 MWh - band IC    3501
Name: count, dtype: int64

In [8]:
energy_df["product"].value_counts().head(10)

product
Electrical energy    3501
Name: count, dtype: int64

In [9]:
energy_df["unit"].value_counts().head(10)

unit
Kilowatt-hour    3501
Name: count, dtype: int64

In [10]:
energy_df = energy_df[energy_df["geo"] == "Germany"]

In [11]:
energy_df = energy_df[energy_df["nrg_cons"] != "HOUSEHOLD"]

In [12]:
energy_df["product"].unique()

array(['Electrical energy'], dtype=object)

In [13]:
energy_df = energy_df[
    energy_df["product"].str.contains("ELECTRIC", case=False) |
    energy_df["product"].str.contains("GAS", case=False)
]

In [14]:
energy_df["unit"].value_counts()

unit
Kilowatt-hour    90
Name: count, dtype: int64

In [15]:
energy_df = energy_df[energy_df["unit"].str.contains("I", case=False)]

In [16]:
energy_df = energy_df[["TIME_PERIOD", "product", "OBS_VALUE"]]

energy_df = energy_df.rename(
    columns={
        "TIME_PERIOD": "year",
        "product": "energy_type",
        "OBS_VALUE": "price_index"
    }
)

In [17]:
energy_df["year"].head()

70    2020-S2
71    2021-S1
72    2021-S2
73    2022-S1
74    2022-S2
Name: year, dtype: object

In [18]:
def convert_period_to_date(period):
    year, half = period.split("-")

    if half == "S1":
        return f"{year}-01-01"
    elif half == "S2":
        return f"{year}-07-01"
    else:
        return None

In [19]:
energy_df["date"] = energy_df["year"].apply(convert_period_to_date)

In [20]:
energy_df["date"] = pd.to_datetime(energy_df["date"])

In [21]:
energy_df = energy_df.drop(columns=["year"])

In [22]:
energy_df.head()
energy_df.shape
energy_df["energy_type"].value_counts()

energy_type
Electrical energy    90
Name: count, dtype: int64

In [23]:
energy_wide = energy_df.pivot_table(
    index="date",
    columns="energy_type",
    values="price_index",
    aggfunc="mean"
)

In [24]:
energy_wide = energy_wide.rename(
    columns={
        col: col.lower() + "_price_index"
        for col in energy_wide.columns
    }
)

In [25]:
energy_wide.head()

energy_type,electrical energy_price_index
date,
2020-07-01,0.157644
2021-01-01,0.160100
2021-07-01,0.165133
2022-01-01,0.201744
2022-07-01,0.205033


In [26]:
energy_wide.shape

(10, 1)

In [27]:
energy_wide.columns = ["electricity_price_index"]

In [28]:
energy_wide.columns = ["electricity_price_index"]

In [29]:
energy_wide.head()

,electricity_price_index
date,
2020-07-01,0.157644
2021-01-01,0.160100
2021-07-01,0.165133
2022-01-01,0.201744
2022-07-01,0.205033


In [30]:
energy_wide.columns

Index(['electricity_price_index'], dtype='object')

In [31]:
energy_wide.to_csv("../data/processed/energy_price_index.csv")